In [9]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [10]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 1.528675328 GB
Current GPU Memory Cached: 3.634364416 GB


In [11]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [12]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaConfig
from transformers.modeling_outputs import SequenceClassifierOutput

class RobertaLSTM(nn.Module):
    def __init__(self, num_labels=2, hidden_size=768, lstm_hidden_dim=128, num_layers=1, bidirectional=True, dropout_prob=0.1):
        super(RobertaLSTM, self).__init__()
        
        # 載入 RobertaModel (而非 RobertaForSequenceClassification)
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        
        # 設定是否雙向、LSTM hidden 大小
        self.lstm_hidden_dim = lstm_hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # LSTM 輸入的維度會是 RoBERTa 輸出的 hidden_size (base 預設768)
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=lstm_hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )
        
        # dropout
        self.dropout = nn.Dropout(dropout_prob)
        
        # 計算最終線性層的輸入維度
        # 如果 LSTM 是雙向 (bidirectional=True)，hidden 維度就會 x2
        lstm_output_dim = lstm_hidden_dim * 2 if bidirectional else lstm_hidden_dim
        
        # 最終輸出 num_labels 個維度，用於做 classification
        self.classifier = nn.Linear(lstm_output_dim, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # 1. 先將 input_ids, attention_mask 丟進 RoBERTa，拿到最後一層 hidden states
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # outputs 裡面有:
        # - last_hidden_state: [batch_size, sequence_length, hidden_size]
        # - pooler_output（如有）
        # 我們需要的是最後一層的 hidden states
        last_hidden_state = outputs.last_hidden_state  # [B, T, 768]
        
        # 2. 透過 LSTM 處理
        #    lstm_out: [B, T, lstm_hidden_dim * 2（若 bidirectional）]
        #    (h_n, c_n): 分別是最終多層 LSTM 的 hidden, cell state
        lstm_out, (h_n, c_n) = self.lstm(last_hidden_state)
        
        # 3. 我們可以取 bidirectional LSTM 的最後一層隱狀態 h_n 來做分類，也可以取最後 time step 的輸出等等
        #    h_n shape = [num_layers * num_directions, batch_size, lstm_hidden_dim]
        #    通常若是雙向且只有1層，最後一層的 forward hidden 在 h_n[-2], backward hidden 在 h_n[-1].
        #    這邊示範將 forward hidden 和 backward hidden concat 作為最終表示。
        if self.bidirectional:
            # 假設只有 1 層 LSTM: h_n[-2] = forward, h_n[-1] = backward
            # 若有多層可以再自行調整
            forward_hidden = h_n[-2,:,:]  # [batch_size, lstm_hidden_dim]
            backward_hidden = h_n[-1,:,:] # [batch_size, lstm_hidden_dim]
            final_hidden = torch.cat((forward_hidden, backward_hidden), dim=1)  # [batch_size, lstm_hidden_dim*2]
        else:
            # 單向 LSTM 直接取 h_n[-1]
            final_hidden = h_n[-1,:,:]    # [batch_size, lstm_hidden_dim]
        
        # dropout
        final_hidden = self.dropout(final_hidden)
        
        # 4. 通過最終線性分類層
        logits = self.classifier(final_hidden)  # [batch_size, num_labels]
        
        # 5. 若有 labels 則計算 loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        
        # 最後回傳 SequenceClassifierOutput（transformers 的標準輸出格式之一）
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )


In [15]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import KFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [16]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
from collections import Counter
import time  # 加入這行來使用時間模組
total_start_time = time.time()  # 記錄整體開始時間

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(kf.split(messages)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
    
    train_dataset = CustomDataset(train_messages, train_labels, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = RobertaLSTM(num_labels=NUM_LABELS, 
                     hidden_size=768,      # RoBERTa-base 預設輸出768
                     lstm_hidden_dim=128,  # 可自行調整
                     num_layers=1,
                     bidirectional=True,
                     dropout_prob=0.1
                    )
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

total_end_time = time.time()
print(f"\nTotal training time: {total_end_time - total_start_time:.2f} seconds")

Fold 1
Training set label distribution: {0: 3585, 1: 28, 2: 123, 3: 26, 4: 239, 5: 134, 6: 101, 7: 37, 8: 80, 9: 70, 10: 82, 11: 199, 12: 209, 13: 34, 14: 25, 15: 31, 16: 22, 17: 258, 18: 106, 19: 47, 20: 15, 21: 39, 22: 89, 23: 183, 24: 15, 25: 55, 26: 100, 27: 52, 28: 13, 29: 15, 30: 17, 31: 89, 32: 39, 33: 36, 34: 29, 35: 13, 36: 32, 37: 24, 38: 5, 39: 118, 40: 10, 41: 10, 42: 166, 43: 18, 44: 23, 45: 11, 46: 58, 47: 601, 48: 215, 49: 82}
Validation set label distribution: {0: 389, 1: 3, 2: 17, 3: 4, 4: 18, 5: 11, 6: 17, 7: 2, 8: 5, 9: 11, 10: 6, 11: 16, 12: 32, 13: 2, 14: 4, 16: 3, 17: 31, 18: 10, 19: 9, 20: 3, 21: 5, 22: 8, 23: 17, 24: 5, 25: 8, 26: 7, 27: 9, 28: 2, 29: 1, 30: 2, 31: 11, 32: 6, 33: 4, 34: 8, 35: 2, 36: 3, 37: 3, 38: 2, 39: 11, 40: 4, 41: 1, 42: 18, 43: 2, 44: 2, 46: 10, 47: 60, 48: 37, 49: 5}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [03:30<00:00,  2.26it/s]


{'val_loss': 1.9378100071313247, 'val_accuracy': 0.541371158392435, 'precision': 0.31750222961196717, 'recall': 0.541371158392435, 'f1-score': 0.39889517252269974, 'macro_f1-score': 0.046850836712406176, 'balanced_accuracy': 0.062048900007064034, 'mcc': 0.3180048045818436}
Epoch 2/20


...

{'val_loss': 1.9201112603241544, 'val_accuracy': 0.493491124260355, 'precision': 0.33598145537788154, 'recall': 0.493491124260355, 'f1-score': 0.3940583043127789, 'macro_f1-score': 0.07396174558509443, 'balanced_accuracy': 0.08512505512388041, 'mcc': 0.29819653890858244}
Epoch 15/20


100%|██████████| 476/476 [03:33<00:00,  2.23it/s]


{'val_loss': 1.492184673840145, 'val_accuracy': 0.6130177514792899, 'precision': 0.5380102557776254, 'recall': 0.6130177514792899, 'f1-score': 0.5603841074076662, 'macro_f1-score': 0.21727264642062138, 'balanced_accuracy': 0.22781117060018255, 'mcc': 0.49978021443908155}
Epoch 16/20


100%|██████████| 476/476 [03:33<00:00,  2.23it/s]


{'val_loss': 1.528665266509326, 'val_accuracy': 0.6011834319526628, 'precision': 0.53714090610432, 'recall': 0.6011834319526628, 'f1-score': 0.5608506015104499, 'macro_f1-score': 0.2110468944649207, 'balanced_accuracy': 0.24042687133500829, 'mcc': 0.4898362607202441}
Early stopping triggered at epoch 16
Final 10-fold Cross-Validation Results:
val_loss: 1.5547001556266138
val_accuracy: 0.6125645347978559
precision: 0.5259690234638202
recall: 0.6125645347978559
f1-score: 0.5495522741235579
macro_f1-score: 0.17030389084008696
balanced_accuracy: 0.19002530161664183
mcc: 0.46836167474482904

Total training time: 36364.81 seconds


In [18]:
all_fold_results

[{'val_loss': 1.5621802367450734,
  'val_accuracy': 0.601485984464708,
  'precision': 0.538815600971519,
  'recall': 0.601485984464708,
  'f1-score': 0.5470336786482708,
  'macro_f1-score': 0.19889485552626093,
  'balanced_accuracy': 0.2215779776015899,
  'mcc': 0.46568690558428144},
 {'val_loss': 1.4903662571086074,
  'val_accuracy': 0.6177895981087471,
  'precision': 0.5339316170580475,
  'recall': 0.6177895981087471,
  'f1-score': 0.5567263184136497,
  'macro_f1-score': 0.1601402175173359,
  'balanced_accuracy': 0.17938356168702202,
  'mcc': 0.4695429231289953},
 {'val_loss': 1.477977532454116,
  'val_accuracy': 0.6476802600472813,
  'precision': 0.5743181241867243,
  'recall': 0.6476802600472813,
  'f1-score': 0.5951872841115864,
  'macro_f1-score': 0.1858946737844318,
  'balanced_accuracy': 0.20834529983175049,
  'mcc': 0.5039292804668943},
 {'val_loss': 1.5462757047735594,
  'val_accuracy': 0.625979843225084,
  'precision': 0.5512758730119467,
  'recall': 0.625979843225084,
  'f1